In [2]:
import pandas as pd
import os, sys
sys.path.append(r'C:\Users\charl\Documents\GitHub\GOST_PublicGoods\GOSTNets\GOSTNets')
sys.path.append(r'C:\Users\charl\Documents\GitHub\GOST')
import GOSTnet as gn
import importlib
importlib.reload(gn)
import geopandas as gpd
import rasterio
from rasterio import features
from shapely.wkt import loads
import numpy as np

peartree version: 0.6.0 
networkx version: 2.2 
matplotlib version: 2.2.2 
osmnx version: 0.8.2 
peartree version: 0.6.0 
networkx version: 2.2 
matplotlib version: 2.2.2 
osmnx version: 0.8.2 


In [144]:
basepth = r'C:\Users\charl\Documents\GOST\Yemen'
pth = r'C:\Users\charl\Documents\GOST\Yemen\graphtool'
net_pth = pth
pckle = r'conflict_adj.pickle'
walk_speed = 4 
WGS = {'init':'epsg:4326'}
measure_crs = {'init':'epsg:32638'}
date = 'January24th'
subset = r'HeRAMS Hospitals %s' % date
OD_name = r'outputOD_%s.csv' % date

### Import All-Destination OD

In [145]:
OD = pd.read_csv(os.path.join(pth, OD_name))
OD = OD.rename(columns = {'Unnamed: 0':'O_ID'})
OD = OD.set_index('O_ID')
OD = OD.replace([np.inf, -np.inf], np.nan)

In [146]:
OD_original = OD
print(OD_original.shape)

(36158, 4114)


### Optional: Subset to Accepted Nodes

In [147]:
acceptable_df = pd.read_csv(os.path.join(pth, 'HeRAMs_2016v2018_damages and functionality_cleaned_snapped.csv'))
print(acceptable_df.columns)
hosp_types = ['District L Rural\r Hospital','Governorate/General\r Hospital','Hospital','1']
def hospcheck(x, hosp_types):
    if x in hosp_types:
        return 'HOS'
    else:
        return 'PHC'
acceptable_df['hosp'] = acceptable_df['Facility Type2'].apply(lambda x: hospcheck(x, hosp_types))
acceptable_df = acceptable_df.loc[acceptable_df['hosp'] == 'HOS']
function_level = ['1','2',1,2]
acceptable_df = acceptable_df.loc[acceptable_df['Functionality2018'].isin(function_level)]
#service_level = ['1']
#acceptable_df = acceptable_df.loc[acceptable_df['Comprehensive Emergency Obstetric Care (S424)'].isin(service_level)]
len(acceptable_df)

Index(['Unnamed: 0', 'Latitude', 'Longitude', 'Year', 'FacilityID',
       'Governorate', 'District', 'Facility Type', 'Functionality2016',
       'Damaged 2016', 'Name of Health Facility', 'Name of Governor ate',
       'Name of District', 'Facility Type2', 'Damaged2018',
       'Functionality2018', 'Accessibility to HF', 'geometry', 'NN',
       'NN_dist'],
      dtype='object')


229

In [148]:
acceptable_df['geometry'] = acceptable_df['geometry'].apply(loads)
acceptable_gdf = gpd.GeoDataFrame(acceptable_df, geometry = 'geometry', crs = {'init':'epsg:4326'})
accepted_facilities = list(set(list(acceptable_df.NN)))
accepted_facilities_str = [str(i) for i in accepted_facilities]
OD = OD_original[accepted_facilities_str]
acceptable_df.to_csv(os.path.join(basepth,'output_layers','%s.csv' % subset))

In [149]:
print(OD_original.shape)
print(OD.shape)

(36158, 4114)
(36158, 227)


### Add Walk Time from Final Node to Destination

In [150]:
dest_df = acceptable_df[['NN','NN_dist']]

dest_df = dest_df.set_index('NN')

dest_df['NN_dist'] = dest_df['NN_dist'] / 1000 * 3600 / walk_speed

dest_df.index = dest_df.index.map(str)

d_f = OD.transpose()

for i in d_f.columns:
    dest_df[i] = d_f[i]
    
for i in dest_df.columns:
    if i == 'NN_dist':
        pass
    else:
        dest_df[i] = dest_df[i] + dest_df['NN_dist']

dest_df = dest_df.drop('NN_dist', axis = 1)

dest_df = dest_df.transpose()

dest_df['min_time'] = dest_df.min(axis = 1)

### Prep Grid

In [151]:
grid_name = r'origins_1km_snapped.csv'
grid = pd.read_csv(os.path.join(pth, grid_name))
grid = grid.rename(columns = {'NN':'O_ID','NN_dist':'walk_to_road_net'})
grid = grid.set_index(grid['O_ID'])

In [152]:
grid['on_network_time'] = dest_df['min_time']
grid['walk_to_road_net'] = grid['walk_to_road_net'] / 1000 * 3600 / walk_speed 
grid['total_time_net'] = grid['on_network_time'] + grid['walk_to_road_net']

### Calculate Direct Walking Time (no driving!), Compare to Drive Time

In [154]:
grid['PLOT_TIME_SECS'] = grid['total_time_net']

In [155]:
grid['PLOT_TIME_MINS'] = grid['PLOT_TIME_SECS'] / 60

### Burn Raster

In [156]:
rst_fn = os.path.join(pth,'pop18_resampled.tif')
out_fn = os.path.join(basepth,'output_layers','%s.tif' % subset)

# Update metadata
rst = rasterio.open(rst_fn, 'r')
meta = rst.meta.copy()
D_type = rasterio.float64
meta.update(compress='lzw', dtype = D_type, count = 2)

with rasterio.open(out_fn, 'w', **meta) as out:
    with rasterio.open(rst_fn, 'r') as pop:
        
        # this is where we create a generator of geom, value pairs to use in rasterizing
        shapes = ((geom,value) for geom, value in zip(grid.geometry, grid.PLOT_TIME_MINS))

        population = pop.read(1).astype(D_type)
        cpy = population.copy()

        travel_times = features.rasterize(shapes=shapes, fill=0, out=cpy, transform=out.transform)

        out.write_band(1, population)
        out.write_band(2, travel_times)

### Generate Zonal Stats

In [157]:
sys.path.append(r'C:\Users\charl\Documents\GitHub\GOST\GOSTRocks')
#from GOSTRocks.rasterMisc import *
### MODIFIED FUNCTION BELOW!!

utils = r'C:\Users\charl\Documents\GOST\Yemen\util_files'

yemen_shp_name = os.path.join(utils, r'Yemen_bound.shp')
yemen_shp = gpd.read_file(yemen_shp_name)
yemen_shp = yemen_shp.to_crs({'init': 'epsg:4326'})

district_shp_name = os.path.join(utils, r'Yemen_adm2.shp')
district_shp = gpd.read_file(district_shp_name)
district_shp = district_shp.to_crs({'init': 'epsg:4326'})

inraster = out_fn
ras = rasterio.open(inraster, mode = 'r+')
pop = ras.read(1)
tt_matrix = ras.read(2)

resolution = 'national'

if resolution == 'national':
    target_shp = yemen_shp
elif resolution == 'district':
    target_shp = district_shp

## First, add on the total population of the district to each district shape

mask_pop = np.ma.masked_where(pop > (200000), pop).mask

base_pop = zonalStats(target_shp, 
                        inraster, 
                        bandNum = 1,
                        mask_A = mask_pop,
                        reProj = False, 
                        minVal = 0,
                        maxVal = np.inf, 
                        verbose = True, 
                        rastType='N')

cols = ['total_pop','min','max','mean']

temp_df = pd.DataFrame(base_pop, columns = cols)

target_shp['total_pop'] = temp_df['total_pop']
target_shp['total_pop'].loc[target_shp['total_pop'] == -1] = 0

## Now, calculate the population within a range of time thresholds from the destination set
for time_thresh in [30,60,120, 240]:
    
    mask_obj = np.ma.masked_where(tt_matrix > (time_thresh), tt_matrix).mask

    raw = zonalStats(target_shp, 
                        inraster, 
                        bandNum = 1,
                        mask_A = mask_obj,
                        reProj = False, 
                        minVal = 0,
                        maxVal = np.inf, 
                        verbose = True, 
                        rastType='N')

    cols = ['pop_%s' % time_thresh,'min','max','mean']

    temp_df = pd.DataFrame(raw, columns = cols)

    target_shp['pop_%s' % time_thresh] = temp_df['pop_%s' % time_thresh]
    target_shp['pop_%s' % time_thresh].loc[target_shp['pop_%s' % time_thresh] == -1] = 0
    target_shp['frac_%s' % time_thresh] = (target_shp['pop_%s' % time_thresh]) / (target_shp['total_pop']).fillna(0)
    target_shp['frac_%s' % time_thresh].replace([np.inf, -np.inf], 0)
    target_shp['frac_%s' % time_thresh] = target_shp['frac_%s' % time_thresh].fillna(0)
    
# Save to file
                  
if resolution == 'national':
    print(out_fn)
    print(target_shp.frac_30.head(1))
    print(target_shp.frac_60.head(1))
    print(target_shp.frac_120.head(1))
    print(target_shp.frac_240.head(1))
else:
    target_shp['abs_pop_iso'] = target_shp['total_pop'] - target_shp['pop_30']
    target_shp.to_file(os.path.join(basepth, 'output_layers','%s_zonal_%s.shp' % (subset, resolution)), driver = 'ESRI Shapefile')

C:\Users\charl\Documents\GOST\Yemen\output_layers\HeRAMS Hospitals January24th.tif
0    0.397615
Name: frac_30, dtype: float64
0    0.610592
Name: frac_60, dtype: float64
0    0.840795
Name: frac_120, dtype: float64
0    0.944127
Name: frac_240, dtype: float64


### Generate Change Rasters

In [9]:
basepth = r'C:\Users\charl\Documents\GOST\Yemen'
pth = r'C:\Users\charl\Documents\GOST\Yemen\graphtool'
start = ['base',
         ' 2018_November1st',
         ' November8th',
         ' November14th',
         ' November25th',
         ' December17th']

end = [' 2018_November1st',
    ' November8th',
    ' November14th',
    ' November25th',
    ' December17th',
    ' January24th']

for i in range(0, len(end)):
    st = start[i]
    en = end[i]
    
    if st == 'base':
        stn = ''
    else:
        stn = st
    
    subset = r'HeRAMS Hospitals_{}_{}'.format(st, en)
    pre_raster = os.path.join(basepth, 'output_layers','HeRAMS Hospitals{}.tif'.format(stn))
    post_raster = os.path.join(basepth, 'output_layers','HeRAMS Hospitals{}.tif'.format(en))
    out_fn = os.path.join(basepth,'output_layers','%s.tif' % subset)

    pre = rasterio.open(pre_raster, 'r')
    arr_pre = pre.read(2)
    post = rasterio.open(post_raster, 'r')
    arr_post = post.read(2)
    delta = arr_pre - arr_post

    # Update metadata
    rst_fn = os.path.join(pth,'pop18_resampled.tif')
    rst = rasterio.open(rst_fn, 'r')
    meta = rst.meta.copy()
    D_type = rasterio.float64
    meta.update(compress='lzw', dtype = D_type, count = 3)

    with rasterio.open(out_fn, 'w', **meta) as out:
        with rasterio.open(rst_fn, 'r') as pop:

            # this is where we create a generator of geom, value pairs to use in rasterizing
            #shapes = ((geom,value) for geom, value in zip(grid.geometry, grid.PLOT_TIME_MINS))

            population = pop.read(1).astype(D_type)

            out.write_band(1, population)
            out.write_band(2, delta)
            out.write_band(3, delta * population)

In [158]:
def zonalStats(inShp, inRaster, bandNum=1, mask_A = None, reProj = False, minVal = '', maxVal = '', verbose=False , rastType='N', unqVals=[]):
    import sys, os, inspect, logging, json
    import rasterio, affine

    import pandas as pd
    import geopandas as gpd
    import numpy as np

    from collections import Counter
    from shapely.geometry import box
    from affine import Affine
    from rasterio import features
    from rasterio.mask import mask
    from rasterio.features import rasterize
    from rasterio.warp import reproject, Resampling
    from osgeo import gdal
    
    ''' Run zonal statistics against an input shapefile
    
    INPUT VARIABLES
    inShp [string or geopandas object] - path to input shapefile
    inRaster [string or rasterio object] - path to input raster
    
    OPTIONAL
    bandNum [integer] - band in raster to analyze
    reProj [boolean] -  whether to reproject data to match, if not, raise an error
    minVal [number] - if defined, will only calculation statistics on values above this number
    verbose [boolean] - whether to be loud with responses
    rastType [string N or C] - N is numeric and C is categorical. Categorical returns counts of numbers
    unqVals [array of numbers] - used in categorical zonal statistics, tabulates all these numbers, will report 0 counts
    mask_A [numpy boolean mask] - mask the desired band using an identical shape boolean mask. Useful for doing conditional zonal stats
    
    RETURNS
    array of arrays, one for each feature in inShp
    '''   
    if isinstance(inShp, str):
        inVector = gpd.read_file(inShp) 
    else:
        inVector = inShp
    if isinstance(inRaster, str):
        curRaster = rasterio.open(inRaster, 'r+')
    else:
        curRaster = inRaster
        
    # If mask is not none, apply mask 
    if mask_A is not None:
        
        curRaster.write_mask(np.invert(mask_A))
    
    outputData=[]
    if inVector.crs != curRaster.crs:
        if reProj:
            inVector = inVector.to_crs(curRaster.crs)
        else:
            raise ValueError("Input CRS do not match")
    fCount = 0
    tCount = len(inVector['geometry'])
    #generate bounding box geometry for raster bbox
    b = curRaster.bounds
    rBox = box(b[0], b[1], b[2], b[3])
    for geometry in inVector['geometry']:
        #This test is used in case the geometry extends beyond the edge of the raster
        #   I think it is computationally heavy, but I don't know of an easier way to do it
        if not rBox.contains(geometry):
            geometry = geometry.intersection(rBox)            
        try:
            fCount = fCount + 1
            if fCount % 1000 == 0 and verbose:
                tPrint("Processing %s of %s" % (fCount, tCount) )
            # get pixel coordinates of the geometry's bounding box
            ul = curRaster.index(*geometry.bounds[0:2])
            lr = curRaster.index(*geometry.bounds[2:4])
            '''
            TODO: There is a problem with the indexing - if the shape falls outside the boundaries, it errors
                I want to change it to just grab what it can find, but my brain is wrecked and I cannot figure it out
            print(geometry.bounds)
            print(curRaster.shape)
            print(lr)
            print(ul)
            lr = (max(lr[0], 0), min(lr[1], curRaster.shape[1]))
            ul = (min(ul[0], curRaster.shape[0]), min(ul[1]))
            '''
            # read the subset of the data into a numpy array
            window = ((float(lr[0]), float(ul[0]+1)), (float(ul[1]), float(lr[1]+1)))
            
            if mask is not None:
                data = curRaster.read(bandNum, window=window, masked = True)
            else:
                data = curRaster.read(bandNum, window=window, masked = False)
            
            # create an affine transform for the subset data
            t = curRaster.transform
            shifted_affine = Affine(t.a, t.b, t.c+ul[1]*t.a, t.d, t.e, t.f+lr[0]*t.e)

            # rasterize the geometry
            mask = rasterize(
                [(geometry, 0)],
                out_shape=data.shape,
                transform=shifted_affine,
                fill=1,
                all_touched=False,
                dtype=np.uint8)

            # create a masked numpy array
            masked_data = np.ma.array(data=data, mask=mask.astype(bool))
            if rastType == 'N':                
                if minVal != '' or maxVal != '':
                    if minVal != '':
                        masked_data = np.ma.masked_where(masked_data < minVal, masked_data)
                    if maxVal != '':
                        masked_data = np.ma.masked_where(masked_data > maxVal, masked_data)                    
                    if masked_data.count() > 0:                        
                        results = [masked_data.sum(), masked_data.min(), masked_data.max(), masked_data.mean()]
                    else :
                        results = [-1, -1, -1, -1]                
                else:
                    results = [masked_data.sum(), masked_data.min(), masked_data.max(), masked_data.mean()]
            if rastType == 'C':
                if len(unqVals) > 0:                          
                    xx = dict(Counter(data.flatten()))
                    results = [xx.get(i, 0) for i in unqVals]                
                else:
                    results = np.unique(masked_data, return_counts=True)                    
            outputData.append(results)
        except Exception as e: 
            print(e)
            outputData.append([-1, -1, -1, -1])            
    return outputData   